In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import ast
import tensorflow as tf
from collections import deque
from keras.models import load_model
from tensorflow.keras.layers import MultiHeadAttention
import mediapipe as mp

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [3]:
def calculate_com(landmarks):
    """
    Calcula el centro de masa (promedio de las posiciones de las articulaciones clave).
    Utiliza las caderas y hombros para el cálculo.
    """
    # Puntos clave: Caderas y Hombros
    left_hip = np.array([landmarks[mp_pose.PoseLandmark.LEFT_HIP].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_HIP].y,
                         landmarks[mp_pose.PoseLandmark.LEFT_HIP].z])

    right_hip = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y,
                          landmarks[mp_pose.PoseLandmark.RIGHT_HIP].z])

    left_shoulder = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x,
                              landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y,
                              landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].z])

    right_shoulder = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x,
                               landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y,
                               landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].z])

    # Calcula el centro de masa como el promedio de los puntos clave
    com = (left_hip + right_hip + left_shoulder + right_shoulder) / 4
    return com


def calculate_torso_orientation(landmarks):
    """
    Calcula la orientación del torso usando los hombros y caderas.
    """
    # Puntos clave: Hombros y Caderas
    left_shoulder = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x,
                              landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y,
                              landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].z])

    right_shoulder = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x,
                               landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y,
                               landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].z])

    left_hip = np.array([landmarks[mp_pose.PoseLandmark.LEFT_HIP].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_HIP].y,
                         landmarks[mp_pose.PoseLandmark.LEFT_HIP].z])

    right_hip = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y,
                          landmarks[mp_pose.PoseLandmark.RIGHT_HIP].z])

    # Vector hombros (dirección entre los hombros)
    shoulder_vector = right_shoulder - left_shoulder

    # Centro de las caderas
    hip_center = (left_hip + right_hip) / 2

    # Vector torso (dirección entre los hombros y el centro de las caderas)
    torso_vector = hip_center - (left_shoulder + right_shoulder) / 2

    # Producto cruzado para obtener la normal al plano del torso
    torso_normal = np.cross(shoulder_vector, torso_vector)

    # Normalización del vector normal
    torso_normal = torso_normal / np.linalg.norm(torso_normal)

    return torso_normal


In [5]:
model = load_model('../fall_detection_model.keras')

In [6]:
WINDOW_SIZE = 130
features_dq = deque(maxlen=WINDOW_SIZE)
lands_dq = deque(maxlen=WINDOW_SIZE)

# Video en vivo (cámara)
cap = cv2.VideoCapture(0)

fps = cap.get(cv2.CAP_PROP_FPS)
frames_per_second = 6
frame_skip = int(fps // frames_per_second)
frame_count = 0

# Variables para velocidad y aceleración
prev_landmarks = None
prev_velocity = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1
    if frame_count % frame_skip != 0:
        continue  # Saltar este frame
    frame_count = 0
    # Procesamiento de imagen
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_world_landmarks:
        landmarks = results.pose_world_landmarks.landmark
        joint_index = 0  # Usualmente la cadera o nariz (ajusta según tu entrenamiento)
        flat_pose = []
        if prev_landmarks is not None:
            x, y, z = landmarks[joint_index].x, landmarks[joint_index].y, landmarks[joint_index].z
            x_prev, y_prev, z_prev = prev_landmarks[joint_index]

            dx, dy, dz = x - x_prev, y - y_prev, z - z_prev
            displacement = np.sqrt(dx**2 + dy**2 + dz**2)

            velocity_x, velocity_y, velocity_z = dx, dy, dz
            velocity = np.sqrt(velocity_x**2 + velocity_y**2 + velocity_z**2)

            if prev_velocity is not None:
                acceleration_x = velocity_x - prev_velocity[0]
                acceleration_y = velocity_y - prev_velocity[1]
                acceleration_z = velocity_z - prev_velocity[2]
                acceleration = np.sqrt(acceleration_x**2 + acceleration_y**2 + acceleration_z**2)
            else:
                acceleration = 0

            tangent_angle = np.arctan2(dy, dx) * (180 / np.pi)
            com = calculate_com(landmarks)
            torso_normal = calculate_torso_orientation(landmarks)

            land = [(lm.x, lm.y, lm.z) for lm in landmarks]
            features = [
                displacement,
                velocity,
                acceleration,
                tangent_angle,
                com[0], com[1], com[2],
                torso_normal[0], torso_normal[1], torso_normal[2]
            ]
            
            for x, y, z in land:
                flat_pose.extend([x, y, z])
            lands_dq.append(flat_pose)
            features_dq.append(features)
            prev_velocity = (velocity_x, velocity_y, velocity_z)
        #print(frame_features_buffer)
            # Solo predecir si tenemos suficiente historial
        if len(lands_dq) == WINDOW_SIZE:
            #input_array = np.array(frame_features_buffer)[None, :, :]  # Shape: (1, 30, 10)
            combined_seq = [f + l for f, l in zip(features_dq, lands_dq)]
            input_array = np.array(list(combined_seq), dtype=np.float32)
            input_array = np.expand_dims(input_array, axis=0)
            #print(input_array.shape)
            prediction = model.predict(input_array)
            predicted_label = np.argmax(prediction)

            label = "Caída" if predicted_label == 1 else "Normal"
            print(f"Predicción: {label} (confianza: {prediction[0][predicted_label]:.2f})")

            # Mostrar en pantalla
            cv2.putText(frame, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.2,
                        (0, 0, 255) if predicted_label == 1 else (0, 255, 0), 3)
            """
        elif len(frame_features_buffer) >= 60:
            num_pose_dims = len(frame_features_buffer[0]) if frame_features_buffer else 130  # 33 puntos x 3 coords
            padding_poses = [[0.0] * num_pose_dims] * (WINDOW_SIZE - len(frame_features_buffer))
            frame_features_buffer.extend(padding_poses)
            """
            

        prev_landmarks = [(lm.x, lm.y, lm.z) for lm in landmarks]

    # Mostrar el video
    cv2.imshow("Detección de Caídas", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # Presiona ESC para salir
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 532ms/step
Predicción: Normal (confianza: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Predicción: Normal (confianza: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Predicción: Normal (confianza: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Predicción: Normal (confianza: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Predicción: Normal (confianza: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Predicción: Normal (confianza: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Predicción: Normal (confianza: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Predicción: Normal (confianza: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Predicción: Normal (confianza: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Predicción: Normal (confianza: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Predicción: Normal (confianza: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Predicción: Normal (confianza: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Predicción: Normal (confianza: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━

In [11]:
df = pd.read_csv("../combined_padded.csv")
# Asegúrate de que los campos 'features' y 'poses_sec' son listas
def safe_literal_eval(val):
    # Maneja valores nulos/faltantes devolviendo lista vacía o None según prefieras
    if pd.isna(val):
        return [] # O manejar de otra forma si es necesario
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except (ValueError, SyntaxError):
            print(f"Advertencia: No se pudo evaluar: {val}")
            return [] # O manejar de otra forma
    return val # Si ya es una lista/otro tipo
df['features'] = df['features'].apply(safe_literal_eval)
df['poses_sec'] = df['poses_sec'].apply(safe_literal_eval)
#df = df[df['video_filename'] == '03_4 (10).mp4']
df

,video_filename,poses_sec,features,target
0,01.mp4,"[[-0.12032114714384079, -0.09099674969911575, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0
1,02.mp4,"[[-0.1629936546087265, -0.0029338218737393618,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0
2,03.mp4,"[[-0.08549799025058746, -0.08539760112762451, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0
3,03_3 (1).mp4,"[[0.5821239948272705, 0.0448436439037323, 0.04...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0
4,03_3 (10).mp4,"[[-0.20032495260238647, -0.5383336544036865, 0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0
...,...,...,...,...
143,21.mp4,"[[-0.1526176780462265, -0.5877647399902344, -0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1
144,22.mp4,"[[0.13059432804584503, -0.5530914068222046, -0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1
145,23.mp4,"[[-0.38498595356941223, -0.2763119339942932, 0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1
146,24.mp4,"[[-0.048113685101270676, -0.42087438702583313,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1


In [9]:
for i, (feats, lms) in enumerate(zip(df['features'], df['poses_sec'])):
    if not feats or not lms:
        print(f"Advertencia: Lista vacía encontrada en índice {i}. Saltando muestra.")
        continue # O manejar añadiendo ceros si es apropiado

    # Verifica si las longitudes coinciden antes de hacer zip
    if len(feats) != len(lms):
        print(f"Advertencia: Longitudes no coinciden en índice {i}. feats: {len(feats)}, lms: {len(lms)}. Usando la longitud menor.")
        min_len = min(len(feats), len(lms))
        feats = feats[:min_len]
        lms = lms[:min_len]

    combined_seq = [f + l for f, l in zip(feats, lms)]
    print(combined_seq)
    print(len(combined_seq))
    input_array = np.array(list(combined_seq), dtype=np.float32)
    input_array = np.expand_dims(input_array, axis=0)
    print(input_array.shape)

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.030468689277768135, -0.40689268708229065, -0.44276759028434753, -0.023319702595472336, -0.45015379786491394, -0.44030654430389404, -0.023016203194856644, -0.4503638446331024, -0.43952813744544983, -0.02272222563624382, -0.4509456157684326, -0.4397238790988922, -0.05457213148474693, -0.4427860975265503, -0.4340522587299347, -0.054065633565187454, -0.44366908073425293, -0.434935063123703, -0.054327089339494705, -0.44520020484924316, -0.4340040981769562, 0.03763314336538315, -0.4747928977012634, -0.3442543148994446, -0.09636180102825165, -0.4567272663116455, -0.3214653432369232, 0.001510334899649024, -0.3997372090816498, -0.4046633541584015, -0.03929423168301582, -0.3916938900947571, -0.3963514566421509, 0.1474524438381195, -0.3470624089241028, -0.22441153228282928, -0.15617358684539795, -0.3470657467842102, -0.21614165604114532, 0.15612900257110596, -0.11274392157793045, -0.24989046156406403, -0.19017647206783295, -0.1068344339728355

In [10]:
prediction = model.predict(input_array)
predicted_label = np.argmax(prediction)
print(prediction)
label = "Caída" if predicted_label == 1 else "Normal"
print(f"Predicción: {label} (confianza: {prediction[0][predicted_label]:.2f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
[[0.05640214 0.9435979 ]]
Predicción: Caída (confianza: 0.94)
